# Input

In [63]:
import pandas as pd
import numpy as np
import pandas_profiling as pp
import warnings
warnings.filterwarnings('ignore')
import xgboost
import lightgbm as lgm
import catboost

In [64]:
train = pd.read_csv('train.csv' , parse_dates= ['date_created', 'user_date_created'] )
test = pd.read_csv('test.csv' ,parse_dates= ['date_created', 'user_date_created'] )

# Look

In [65]:
train.shape , test.shape

((1281499, 11), (252099, 11))

In [66]:
(train['date_created'].max() - train['date_created'].min() ).days , (test['date_created'].max() - test['date_created'].min() ).days

(2813, 145)

In [81]:
train.head()

,comment_id,company_id,user_id,product_id,comment,rating,date_created,user_date_created,user_ip,user_agent,is_fake
0,1,2169053,91933,6.0,компания не выполнила заказ без обьяснения пр...,1,1709,0,150536,62271,0
1,2,709306,91933,2.0,"Впервые встретился такой професиональный, отве...",5,2487,0,745089,30696,0
2,3,35287,91933,NaN,"Несерйозная, некоректная менеджер інтернет маг...",1,2550,0,380703,41360,0
3,4,1778322,91933,1.0,"Не рекомендую. Не отвечают, не выполняют заказы.",1,2685,0,380968,41363,0
4,5,2821533,91933,NaN,"На письма не отвечают, заказы не выполняют. Не...",1,2695,0,385739,41364,0


In [73]:
test

,comment,comment_id,company_id,date_created,month_created,product_id,rating,user_agent,user_date_created,user_id,user_ip
0,"спасибо все отлично, качество как на картинке,...",1655822,2214161,2952,2018-03,1301549.0,5,NaN,2827,12969242,NaN
1,Заказывала шоколад Риттер Спорт 10 шт. К сожал...,1655823,2333068,2852,2017-12,699700.0,5,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,2827,12969248,91.90.13.210
2,"Быстро отзвонились, охотно отвечали на все воп...",1655824,752692,2813,2017-11,NaN,5,Mozilla/5.0 (Windows NT 6.1; rv:38.0) Gecko/20...,2827,12969251,93.115.95.204
3,Очень оперативный и внимательный к покупателю ...,1655825,334751,2833,2017-11,1279247.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2827,12969251,37.28.153.28
4,Все прошло очень оперативно! Заказ привезли в ...,1655826,2263650,2833,2017-11,1225545.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2827,12969251,37.28.153.28
5,Отличная компания. Очень оперативно и качестве...,1655827,1958752,2833,2017-11,1299459.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2827,12969251,37.28.153.28
6,Все четко. Заказал наложенным платежом. Пришел...,1655828,2840325,2833,2017-11,1301564.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2827,12969251,37.28.153.28
7,"Компетентность, оперативность, пунктуальность....",1655829,841233,2833,2017-11,1222815.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2827,12969251,37.28.153.28
8,Такой быстрой доставки еще не встречал. Все пр...,1655830,1981477,2813,2017-11,NaN,5,Mozilla/5.0 (Windows NT 6.1; rv:38.0) Gecko/20...,2827,12969251,93.115.95.204
9,"Заказ выполнен в срок, адекватная цена, без пр...",1655831,333335,2833,2017-11,1301991.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2827,12969251,37.28.153.28


# Pre

In [75]:
test.drop(columns = 'month_created' , inplace = True)

# Encoding

In [74]:
from collections import defaultdict
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

def __encode_categorical(df_list, cat_cols):
    # initialize placeholder
    d = defaultdict(LabelEncoder)
    # fit and encode train/test,
    codes = pd.concat( [df[cat_cols] for df in df_list] , axis=0 )\
            .fillna('')\
            .apply(lambda x: d[x.name].fit(x)),
            
    # transform encodings to train/test etc
    for df in df_list:
        df[cat_cols] = df[cat_cols].fillna('').apply(lambda x: d[x.name].transform(x))

__encode_categorical([train , test] , ['user_ip' , 'user_agent'])

In [68]:
startTime_date_created = train['date_created'].min()
startTime_user_created = train['user_date_created'].min()

train['date_created'] = [ (date - startTime_date_created).days  for date in train['date_created']]
test['date_created'] = [ (date - startTime_date_created).days  for date in test['date_created']]

train['user_date_created'] = [ (date - startTime_user_created).days  for date in train['user_date_created']]
test['user_date_created'] = [ (date - startTime_user_created).days  for date in test['user_date_created']]

train['is_fake'] = train['is_fake'].astype(int)

In [80]:
pp.ProfileReport(train)

Number of variables,11
Number of observations,1281499
Total Missing (%),3.2%
Total size in memory,102.7 MiB
Average record size in memory,84.0 B
Numeric,7
Categorical,1
Boolean,1
Date,0
Text (Unique),0
Rejected,2


In [83]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
train_text = train['comment']
test_text = test['comment']
all_text = pd.concat([train_text, test_text])

In [84]:
word_vectorizer = TfidfVectorizer( min_df = 3 , ngram_range = (1 , 1))
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
word_vectorizer

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=3,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

# Val

In [39]:
(train['date_created'].max() - train['date_created'].min() ).days

2813

In [14]:
train.shape[0]/ test.shape[0]

5.083316474876933

In [88]:
from sklearn.model_selection import StratifiedKFold , cross_val_score , GridSearchCV
X = train.drop(columns = 'is_fake')
y = train['is_fake'].astype(int)
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42)

In [36]:
test

,comment,comment_id,company_id,date_created,month_created,product_id,rating,user_agent,user_date_created,user_id,user_ip
0,"спасибо все отлично, качество как на картинке,...",1655822,2214161,2018-03-19 21:36:07,2018-03,1301549.0,5,NaN,2017-11-01 00:00:46,12969242,NaN
1,Заказывала шоколад Риттер Спорт 10 шт. К сожал...,1655823,2333068,2017-12-10 13:46:49,2017-12,699700.0,5,Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...,2017-11-01 00:02:19,12969248,91.90.13.210
2,"Быстро отзвонились, охотно отвечали на все воп...",1655824,752692,2017-11-01 00:07:38,2017-11,NaN,5,Mozilla/5.0 (Windows NT 6.1; rv:38.0) Gecko/20...,2017-11-01 00:02:44,12969251,93.115.95.204
3,Очень оперативный и внимательный к покупателю ...,1655825,334751,2017-11-20 23:40:01,2017-11,1279247.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
4,Все прошло очень оперативно! Заказ привезли в ...,1655826,2263650,2017-11-20 23:39:51,2017-11,1225545.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
5,Отличная компания. Очень оперативно и качестве...,1655827,1958752,2017-11-20 23:40:16,2017-11,1299459.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
6,Все четко. Заказал наложенным платежом. Пришел...,1655828,2840325,2017-11-20 23:39:45,2017-11,1301564.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
7,"Компетентность, оперативность, пунктуальность....",1655829,841233,2017-11-20 23:39:36,2017-11,1222815.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28
8,Такой быстрой доставки еще не встречал. Все пр...,1655830,1981477,2017-11-01 00:07:01,2017-11,NaN,5,Mozilla/5.0 (Windows NT 6.1; rv:38.0) Gecko/20...,2017-11-01 00:02:44,12969251,93.115.95.204
9,"Заказ выполнен в срок, адекватная цена, без пр...",1655831,333335,2017-11-20 23:40:39,2017-11,1301991.0,5,Mozilla/5.0 (Windows NT 6.1; Win64; x64) Apple...,2017-11-01 00:02:44,12969251,37.28.153.28


# Model

In [ ]:
pd.DataFrame(train_word_features)

In [97]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
classifier = LogisticRegression(C =2.3 , #solver ='sag' , n_jobs =-1  
                               ) 
cross_val_score(classifier , train_word_features, y , scoring  = 'neg_log_loss' , verbose =2 , cv =3 , #n_jobs =-1
               )

[CV]  ................................................................
[CV] ................................................. , total=  26.4s
[CV]  ................................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   26.7s remaining:    0.0s


[CV] ................................................. , total=  29.9s
[CV]  ................................................................
[CV] ................................................. , total=  22.1s


[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.3min finished


array([-0.02813651, -0.02060814, -0.01952089])

In [99]:
np.array([-0.02813651, -0.02060814, -0.01952089]).mean()

-0.02275518